<a href="https://colab.research.google.com/github/itimes-digital/mineracao-texto/blob/main/analise_de_texto_para_classificacao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [152]:
import nltk
import pandas as pd
import re
from unicodedata import normalize

In [153]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [154]:
nltk.download('rslp')

[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Package rslp is already up-to-date!


True

### Preparação dos dados

In [155]:
base = pd.read_csv('sample_data/lista_classificada.csv', sep=';', engine='python')

In [156]:
base.head()

,Categoria,Comentario
0,"Cadastro, Login, Senha",Nao recebi email para validar o cadastro NÃO C...
1,"Cadastro, Login, Senha",Não consigo concluir o cadastro.
2,"Loja C&R, Estoque",Nunca realizei compras pelo app. Pois a forma ...
3,Entrega,"Muito demora pra entregar.Farmácia é urgência,..."
4,Entrega,"Péssimo no que se refere ao item ENTREGA, pois..."


In [157]:
base['Comentario'] = base['Comentario'].map(lambda x: re.sub('[,\.!?-]', '', x))
base['Comentario'] = base['Comentario'].map(lambda x: normalize('NFKD', x.lower()).encode('ASCII','ignore').decode('ASCII'))

In [158]:
base.head()

,Categoria,Comentario
0,"Cadastro, Login, Senha",nao recebi email para validar o cadastro nao c...
1,"Cadastro, Login, Senha",nao consigo concluir o cadastro
2,"Loja C&R, Estoque",nunca realizei compras pelo app pois a forma d...
3,Entrega,muito demora pra entregarfarmacia e urgencia q...
4,Entrega,pessimo no que se refere ao item entrega pois ...


In [159]:
base['Categoria'].value_counts()

Lentidão, Trava, Crash, Performance    121
Entrega                                 85
Pedido cancelamento                     39
Erro finalização                        38
Erro PIX                                34
Loja C&R, Estoque                       27
Preço, Prazo, Promoção, Stix            21
Erro cartão de crédito                  11
Cesta                                   10
Busca                                    7
Novas Funcionalidades                    6
Cadastro, Login, Senha                   5
Problema não detalhado                   3
Name: Categoria, dtype: int64

In [160]:
base = base.sample(frac=1)

In [161]:
base_treinamento = base.iloc[:350,:]
base_teste = base.iloc[350:,:]

In [162]:
base_treinamento.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 350 entries, 169 to 196
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Categoria   350 non-null    object
 1   Comentario  350 non-null    object
dtypes: object(2)
memory usage: 8.2+ KB


In [163]:
base_teste.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 57 entries, 129 to 79
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Categoria   57 non-null     object
 1   Comentario  57 non-null     object
dtypes: object(2)
memory usage: 1.3+ KB


In [164]:
base_treinamento = base_treinamento.reset_index(drop=True)
base_teste = base_teste.reset_index(drop=True)

In [165]:
base_teste.head()

,Categoria,Comentario
0,Erro cartão de crédito,nao esta mais aceitando pagamento por cartao s...
1,Entrega,nao esta dando opcao para retirar em loja assi...
2,Busca,dificil de encontrar produtos
3,"Preço, Prazo, Promoção, Stix",precos subiram medicamentos faltando cobranca ...
4,Entrega,tempo de entrega e muito demorado


In [166]:
stopwords = nltk.corpus.stopwords.words('portuguese')
#print(stopwords)

In [167]:
def removeStopWordsAplicandoStemmer(base):
    stemmer = nltk.stem.RSLPStemmer();
    frases = [];
    for i in range(len(base)):
        sem_stop = [str(stemmer.stem(p)) for p in base.loc[i,"Comentario"].split() if p not in stopwords]
        frases.append((sem_stop, base.loc[i,"Categoria"]))
    return frases;

In [168]:
frasesComStemmingTreinamento = removeStopWordsAplicandoStemmer(base_treinamento);
frasesComStemmingTeste = removeStopWordsAplicandoStemmer(base_teste);
#print(frasesComStemming)

In [169]:
def buscaPalavras(frases):
    todasPalavras = [];
    for (palavras, categoria) in frases:
        todasPalavras.extend(palavras);
    return todasPalavras;

In [170]:
palavrasTreinamento = buscaPalavras(frasesComStemmingTreinamento);
palavrasTeste = buscaPalavras(frasesComStemmingTeste);
#print(palavras)

In [171]:
def buscaFrequencia(palavras):
    palavras = nltk.FreqDist(palavras);
    return palavras;

In [172]:
frequenciaTreinamento = buscaFrequencia(palavrasTreinamento);
frequenciaTeste = buscaFrequencia(palavrasTeste);
#print(frequencia.most_common(50))

In [173]:
def buscaPalavrasUnicas(frequencia):
    freq = frequencia.keys();
    return freq;

In [174]:
palavrasUnicasTreinamento = buscaPalavrasUnicas(frequenciaTreinamento);
palavrasUnicasTeste = buscaPalavrasUnicas(frequenciaTeste);
#print(palavrasUnicas)

In [175]:
def extratorPalavras(documento):
    doc = set(documento);
    caracteristicas = {};
    for palavras in palavrasUnicasTreinamento:
        caracteristicas['%s' % palavras] = (palavras in doc);
    return caracteristicas;

In [176]:
caracteristicasFrases = extratorPalavras({'am', 'nov', 'dia'});
#print(caracteristicasFrases)

### Realizando a classificação

In [177]:
baseCompletaTreinamento = nltk.classify.apply_features(extratorPalavras, frasesComStemmingTreinamento);
baseCompletaTeste = nltk.classify.apply_features(extratorPalavras, frasesComStemmingTeste);
#print(baseCompleta);

In [178]:
classificador = nltk.NaiveBayesClassifier.train(baseCompletaTreinamento);

In [179]:
print(classificador.labels())

['Lentidão, Trava, Crash, Performance', 'Cadastro, Login, Senha', 'Cesta', 'Preço, Prazo, Promoção, Stix', 'Entrega', 'Erro PIX', 'Pedido cancelamento', 'Novas Funcionalidades', 'Erro finalização', 'Loja C&R, Estoque', 'Erro cartão de crédito', 'Busca', 'Problema não detalhado']


In [180]:
 #usando a mesma amostra do modelo treinado
print(nltk.classify.accuracy(classificador, baseCompletaTreinamento))

0.6857142857142857


In [181]:
#usando a amostra de teste
print(nltk.classify.accuracy(classificador, baseCompletaTeste)) 

0.5263157894736842


In [182]:
print(classificador.show_most_informative_features(10))

Most Informative Features
                  cancel = True           Pedido : Lentid =     67.1 : 1.0
                     pix = True           Erro P : Lentid =     38.6 : 1.0
                   carta = True           Erro c : Entreg =     37.6 : 1.0
                     peg = True           Proble : Lentid =     35.0 : 1.0
                  adiant = True           Proble : Lentid =     35.0 : 1.0
                   retir = True           Proble : Lentid =     35.0 : 1.0
                   codig = True           Erro P : Lentid =     28.2 : 1.0
                  entreg = True           Entreg : Lentid =     25.3 : 1.0
                    salv = True           Novas  : Lentid =     25.0 : 1.0
                  estoqu = True           Loja C : Lentid =     24.8 : 1.0
None


In [ ]:
erros = []

for (frase, classe) in baseCompletaTeste:
    previsao = classificador.classify(frase)

    if previsao != classe:
        erros.append((classe, previsao, frase))

for (flasse, previsao, frase) in erros:
    # Classe real | previsão da classe | frase
    print(classe, previsao, frase)

In [184]:
from nltk.metrics import ConfusionMatrix

esperado = []
previsto = []

for (frase, classe) in baseCompletaTeste:
    previsao = classificador.classify(frase);
    previsto.append(previsao);
    esperado.append(classe);

matriz = ConfusionMatrix(esperado, previsto)
print(matriz)

                                    |                    L             |
                                    |                    e             |
                                    |                    n             |
                                    |                    t             |
                                    |                    i             |
                                    |                    d             |
                                    |                    ã             |
                                    |                    o        P    |
                                    |                    ,        r    |
                                    |                             e    |
                                    |                    T        ç    |
                                    |                    r        o    |
                                    |                    a        ,    |
                                    |              

### Montagem de uma frase para testar classificação.

In [185]:
testeFrase = 'Meu pedido não fecha adoentado'

In [186]:
def aplicandoStemmerFrase(frase):
    stemmer = nltk.stem.RSLPStemmer();
    frases = [];
    for (palavrasTreinamento) in frase.split():
        com_stem_sem_stop = [p for p in palavrasTreinamento.split()];
        frases.append(str(stemmer.stem(com_stem_sem_stop[0])));
    return frases;

In [187]:
fraseComStemming = aplicandoStemmerFrase(testeFrase)
print(fraseComStemming)

['meu', 'ped', 'não', 'fech', 'adoent']


In [188]:
novaFrase = extratorPalavras(fraseComStemming);
#print(novaFrase)

In [189]:
print(classificador.classify(novaFrase))

Lentidão, Trava, Crash, Performance


In [190]:
distribuicao = classificador.prob_classify(novaFrase)
for classe in distribuicao.samples():
    print("%s: %f" % (classe, distribuicao.prob(classe)))

Lentidão, Trava, Crash, Performance: 0.999854
Cadastro, Login, Senha: 0.000000
Cesta: 0.000000
Preço, Prazo, Promoção, Stix: 0.000000
Entrega: 0.000145
Erro PIX: 0.000000
Pedido cancelamento: 0.000000
Novas Funcionalidades: 0.000000
Erro finalização: 0.000000
Loja C&R, Estoque: 0.000000
Erro cartão de crédito: 0.000000
Busca: 0.000000
Problema não detalhado: 0.000000


### Montagem dos dados novos para classificação

In [191]:
baseSemCategoria = pd.read_csv('sample_data/lista.csv', sep=';', engine='python')

In [192]:
baseSemCategoria['Comentario'] = baseSemCategoria['Comentario'].map(lambda x: re.sub('[,\.!?-]', '', x))
baseSemCategoria['Comentario'] = baseSemCategoria['Comentario'].map(lambda x: normalize('NFKD', x.lower()).encode('ASCII','ignore').decode('ASCII'))

In [193]:
def removeStopWordsAplicandoStemmerSemCategoria(base):
    stemmer = nltk.stem.RSLPStemmer();
    frases = [];
    for i in range(len(base)):
        sem_stop = [str(stemmer.stem(p)) for p in base.loc[i,"Comentario"].split() if p not in stopwords];
        frases.append(sem_stop);
    return frases;

In [194]:
frasesComStemmingSemCategoria = removeStopWordsAplicandoStemmerSemCategoria(baseSemCategoria)
#print(frasesComStemmingSemCategoria)

In [195]:
baseCompletaSemClassificacao = nltk.classify.apply_features(extratorPalavras, frasesComStemmingSemCategoria);

In [196]:
#print(baseCompletaSemClassificacao)